In [5]:
import cv2
import mediapipe as mp
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Conv2D,Activation,Flatten,MaxPool2D,Dropout,BatchNormalization
from sklearn.model_selection import train_test_split
import os
import numpy as np

os.chdir("//home//divs//Documents//Vedeo//")
print(os.getcwd())

/home/divs/Documents/Vedeo


In [3]:
df_train=pd.read_csv("sign_mnist_train.csv")
df_test=pd.read_csv("sign_mnist_test.csv")
df1=pd.DataFrame(df_train)
df2=pd.DataFrame(df_test)

y_train=df1.values[:,:1]
y_test=df2.values[:,:1]
x_train=df1.values[:,1:]
x_test=df2.values[:,1:]

x_train_new=[]
x_test_new=[]


for i in range(len(x_train)):
    image=x_train[i]
    
    image=image.reshape(28,28)
    x_train_new.append(image)
   
for i in range(len(x_test)):
    image=x_test[i]
    
    image=image.reshape(28,28)
    
    x_test_new.append(image)
    
num_classes=26
y_train_new=np.array(y_train).reshape(-1)
y_test_new=np.array(y_test).reshape(-1)
y_train_new=np.eye(num_classes)[y_train_new]
y_test_new=np.eye(num_classes)[y_test_new]
x_test_new=np.array(x_test_new)
x_train_new=np.array(x_train_new)

# print(x_train_new)

In [6]:
print(x_train_new.shape)
print(y_train_new.shape)
    
model=Sequential()
model.add(Conv2D(filters=8,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(28,28,1),activation='sigmoid',data_format='channels_last'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dense(128,activation='relu'))
model.add(Conv2D(filters=8,kernel_size=(3,3),strides=(1,1),padding='same',activation='sigmoid'))
model.add(Dropout(0.2))
model.add(MaxPool2D(pool_size=(4,4)))
model.add(Dense(128,activation='relu'))
model.add(Flatten())
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train_new,y_train_new,epochs=25,batch_size=100)
accuracy=model.evaluate(x=x_test_new,y=y_test_new,batch_size=100)
print(accuracy[1])


(27455, 28, 28)
(27455, 26)
Epoch 1/25


ValueError: in user code:

    File "/home/divs/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/divs/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/divs/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/divs/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/divs/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/home/divs/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, samp

In [5]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 8)         584       
                                                                 
 dropout (Dropout)           (None, 14, 14, 8)         0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 8)          0         
 2D)                                                             
                                                                 
 dense (Dense)               (None, 3, 3, 128)         1

In [6]:
labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# Initialize webca


# Initialize Mediapipe hand detection


while True:
    # Capture frame-by-frame
    cap = cv2.VideoCapture(0)
    ret,frame = cap.read()
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands using Mediapipe
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        # Get the bounding box of the hand
        for landmarks in results.multi_hand_landmarks:
            x_min, x_max, y_min, y_max = 1000, 0, 1000, 0
            for landmark in landmarks.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                if x < x_min:
                    x_min = x
                if x > x_max:
                    x_max = x
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y

            # Extract hand region
            hand_frame = frame[y_min:y_max, x_min:x_max]
            
            if hand_frame.shape[0] > 0 and hand_frame.shape[1] > 0:
                # Preprocess the hand frame
                resized_hand_frame = cv2.resize(hand_frame, (28, 28))
                grayscale_hand_frame = cv2.cvtColor(resized_hand_frame, cv2.COLOR_BGR2GRAY)
                flattened_hand_frame = grayscale_hand_frame.flatten()
                normalized_hand_frame = flattened_hand_frame / 255.0

                # Make a prediction
                input_frame = normalized_hand_frame.reshape(1, 784)
                prediction = model.predict(input_frame)
                predicted_label = np.argmax(prediction)

                # Get the corresponding letter
                letter = labels[predicted_label]

                # Display the prediction on the hand frame
                cv2.putText(hand_frame, letter, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Display both frames in separate windows
                cv2.imshow('Original Frame', frame)
                cv2.imshow('Hand Frame', hand_frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
[ WARN:0@37.978] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@37.979] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
import cv2
  
  
# define a video capture object

while(True):
      
    # Capture the video frame
    # by frame
    vid = cv2.VideoCapture(0)
  
    ret, frame = vid.read()
  
    # Display the resulting frame
    cv2.imshow('frame', frame)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()